<a href="https://colab.research.google.com/github/ucfilho/thesis_advices/blob/master/RTC_DE_desmonta_IF_new_05_fev_22_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x,cols): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x,cols):

  nref=int(len(x)/2) # metade de x é deltaT e a outra metade variavel logica
  matrizDeltaT=np.zeros((cols,cols))
  matrizBinaria=np.zeros((cols,cols))

  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizDeltaT[i,j]=x[contador]
        contador=contador+1
  
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        matrizBinaria[i,j]=round(x[contador+nref])
        contador=contador+1
        
  return matrizDeltaT,matrizBinaria

In [0]:
def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):

  cols=len(mCp)
  cargaTermica=np.zeros((cols,cols))
  contador=0
  for i in range(cols):
    for j in range(cols):
      contador=contador+1
      cargaTermica[i,j]=mCp[i]*matrizDeltaT[i,j]

      if(i<j):
        if(matrizBinaria[i,j]==1):
          if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j
            #i: CQ, j: CF
            tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
            # T1, T2, T3 e T4 sao testes
            T1=(tempAtual[i]>tempAtual[j])
            T2=(tempSaidaQ>tempAtual[j])
            T3=(tempAtual[i]>tempSaidaF)
            T4=(contador < cols)
            Teste1=(T1 and T2 and T3 and T4)

            if(Teste1):
              tempAtual[i]=tempSaidaQ
              tempAtual[j]=tempSaidaF
          else: #if(matrizDeltaT[i,j]<0)
            #a corrente i ganha calor da j
            #i: CF, j: CQ
            tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
            # Ta1, Ta2, Ta3 e Ta4 sao testes alternativos
            Ta1=(tempAtual[i]>tempAtual[j])
            Ta2=(tempSaidaQ>tempAtual[j])
            Ta3=(tempAtual[i]>tempSaidaF)
            Ta4=(contador < cols)
            Teste2=(Ta1 and Ta2 and Ta3 and Ta4)
            if(Teste2):
              tempAtual[i]=tempSaidaF
              tempAtual[j]=tempSaidaQ

  return tempAtual,cargaTermica

SyntaxError: ignored

In [0]:
def Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT):
  cols=len(tempInicial)
  Trocador=np.zeros((cols,cols)) # matriz q armazena as areas dos trocadores
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        Carga=cargaTermica[i,j]
        # calcula a area de placas das duas correntes que trocam i e j

        if(matrizDeltaT[i,j]<0):
          #a corrente i perde calor para a j
          #i: CQ, j:CF
          tempSaidaF=tempAtual[j]
          tempSaidaQ=tempAtual[i]
          try:
            A=((tempSaidaQ-tempSaidaF)-(tempAtual[i]-tempAtual[j]))
            B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[i]-tempAtual[j]))
            deltaTMediaLog=A/B
          except:
            deltaTMediaLog=1e99
        else: # if(matrizDeltaT[i,j]<0):
          #a corrente i ganha calor da j
          #i: CF, j: CQ
          tempSaidaF=tempAtual[i]
          tempSaidaQ=tempAtual[j]
          try:
            A=((tempSaidaQ-tempSaidaF)-(tempAtual[j]-tempAtual[i]))
            B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[j]-tempAtual[i]))
            deltaTMediaLog=A/B
          except:
            deltaTMediaLog=1e99
       
        # parte dentro do if(i<j) que usa o calculo e gera matriz area
        
        if(deltaTMediaLog==1e99):  
          AreaCalc=1e99
        else:
          AreaCalc=abs(Carga/(U*deltaTMediaLog))

        if(tempAtual[i]==tempInicial[i]):
          AreaCalc=0

        if(tempAtual[j]==tempInicial[j]):
          AreaCalc=0

        Trocador[i,j]=AreaCalc

  return Trocador                     

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte

def FUN(x):

  global mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF
  
  cols=len(mCp)
  tempInicial=np.copy(tempAtual) 
  matrizDeltaT,matrizBinaria=buildMatriz(x,cols) # transforma x em matriz logica e troca
  tempAtual,Carga=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
  areaTrocador=Area(U,tempAtual,tempInicial,Carga,matrizDeltaT)
  cargaUQ,cargaUF=GastoUtilidades(tempAlvo,tempAtual,mCp)


  for i in range(cols):
    deltaT=tempAlvo[i]-tempAtual[i]
    if(deltaT>0):#usa UQ
      cargaUQ=cargaUQ+mCp[i]*deltaT
      #UQ -> vapor saturado -> entra e sai a 100ºC
      #A=(373.15-(tempAlvo[i])-(373.15-tempAtual[i]))
      #B=np.log((373.15-tempAlvo[i])/(373.15-tempAtual[i]))
      # deltaTMediaLog=A/B
      try:
        A=(373.15-(tempAlvo[i])-(373.15-tempAtual[i]))
        B=np.log((373.15-tempAlvo[i])/(373.15-tempAtual[i]))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      areaTrocador=areaTrocador+abs(mCp[i]*(deltaT)/(U*deltaTMediaLog))
    else:#usa UF      
      cargaUF=cargaUF+mCp[i]*(-deltaT)
      #UF -> água resfriada -> entra a 5ºC e sai a 20ºC
      try:
        A=((tempAlvo[i]-293.15)-(tempAtual[i]-278.15))
        B=np.log((tempAlvo[i]-293.15)/(tempAtual[i]-278.15))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      areaTrocador=areaTrocador+abs(mCp[i]*(-deltaT)/(U*deltaTMediaLog))
  try:
    costTrocador=300*(areaTrocador)**0.5
  except:
    costTrocador=1e99
  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF
  funr=costUQ+costUF+costTrocador

  print('matrizBinaria');print(matrizBinaria)   
  print('matrizDeltaT');print(matrizDeltaT)  
  print(tempAtual)


  return funr

In [0]:
#MATGRAF retorna o gráfico das trocas térmicas em cada trocador
# function matrizGrafico=MATGRAF(matrizDeltaT,mCp,tempAtual)
def MATGRAF(matrizDeltaT,mCp,tempAtual):
  cols=len(mCp)
  matrizGrafico=np.zeros((cols,cols))
  contador=0
  matrizGrafico[:,0]=tempAtual
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j
          #i: CQ, j:CF
          cargaTermica=mCp[i]*matrizDeltaT[i,j]
          tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
          tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
          if(
              tempAtual[i]>tempAtual[j] and
              tempSaidaQ>tempAtual[j] and
              tempAtual[i]>tempSaidaF and
              contador < cols
              
            ):
            tempAtual[i]=tempSaidaQ
            tempAtual[j]=tempSaidaF
            matrizGrafico[:,contador]=tempAtual
            
            #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
            contador=contador+1
            # troquei o elif por else (fica melhor no contexto?)
            # elif (matrizDeltaT[i,j]>0) : #a corrente i ganha calor da j
      else : #a corrente i ganha calor da j

        # situa
        cargaTermica=mCp[i]*matrizDeltaT[i,j]
        tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
        tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j] 
        if (
          
            tempAtual[i]<tempAtual[j] and 
            tempSaidaF<tempAtual[j] and 
            tempAtual[i]<tempSaidaQ and
            contador< cols

            ):
            #  i: CF, j: CQ
              
            tempAtual[i]=tempSaidaF
            tempAtual[j]=tempSaidaQ
            matrizGrafico[:,contador]=tempAtual
            contador=contador+1          

  return matrizGrafico

In [0]:
#MATBEST retorna a matriz com os deltas das trocas térmicas que ocorreram e a matriz binária
# function [matrizTrocaBest,matrizBinaria]=MATBEST(x,mCp,tempAtual)
def MATBEST(x,mCp,tempAtual):
  
  nref=int(len(x)/2)
  #nref=0
  cols=len(mCp)

  # cada particula contem metade das variaveis para matriz binaria
  # e outra metade para deltaT nref e para separar os dois casos
  matrizDeltaT=np.zeros((cols,cols))
  matrizTrocaBest=np.zeros((cols,cols))
  # contador=-1
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        # contador=contador+1
        matrizDeltaT[i,j]=x[contador]
        contador=contador+1
    
    '''
    nao tem esta linha (contador=0) abaixo 
    no original mas se tirar ela gera out of bounds
    '''
    contador=0 
    matrizBinaria=np.zeros((cols,cols))
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          # matrizBinaria[i,j]=round(x[contador-1])
          '''
          parece q faz mais sentido assim (forma abaixo)
          '''
          matrizBinaria[i,j]=round(x[contador+nref])
          # print('matriz binaria',matrizBinaria[i,j])
          contador=contador+1
  
    tempSaidaQ=0;tempSaidaF=0;cargaTermica=0
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          if(matrizBinaria[i,j]==1):
            if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j 
              #i: CQ, j: CF
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]>tempAtual[j] and
                  tempSaidaQ>tempAtual[j] and
                  tempAtual[i]>tempSaidaF
                ): #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
                tempAtual[i]=tempSaidaQ
                tempAtual[j]=tempSaidaF
              else:
                matrizDeltaT[i,j]=0
            else:#a corrente i ganha calor da j
              # i: CF, j:CQ
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]<tempAtual[j] and
                  tempSaidaF<tempAtual[j] and
                  tempAtual[i]<tempSaidaQ
                 ):
                #TFentra<TQentra & TFsai<TQentra & TFentra<TQsai
                tempAtual[i]=tempSaidaF
                tempAtual[j]=tempSaidaQ
              else:
                matrizDeltaT[i,j]=0
          else:
            matrizDeltaT[i,j]=0
  print('contador=',contador)
  matrizTrocaBest=matrizDeltaT
  return matrizTrocaBest,matrizBinaria

In [0]:
#********************PROGRAMA PRINCIPAL*************************

NPAR=50 #Número de partículas
ITE=100 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

MAX=[100,100,100,100,100,100,1,1,1,1,1,1]
MIN=[-100,-100,-100,-100,-100,-100,0,0,0,0,0,0]

mCp=[4,2,1.5,3] #[kW/K]
tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]
coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

x=[ 74,20,83,-10,38,-50,0,0,0,0,1,1]
resp=FUN(x)
print('===++++++++++======')
print(resp)

x=[ 90,20,83,-10,38,-50,1,1,0,0,1,1]
resp=FUN(x)
print('==xxxxxxxxxx=====')
print(resp)

x=[ -90,20,83,-10,38,-50,1,1,0,0,1,1]
resp=FUN(x)
print('==xxxxxxxxxx=====')
print(resp)

bounds = []
for i in range(12):
  bounds.append((MAX[i],MIN[i]))

for it in bounds:
  print(it)


logica
False False False False
523 533
matrizBinaria
[[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]
matrizDeltaT
[[  0.  74.  20.  83.]
 [  0.   0. -10.  38.]
 [  0.   0.   0. -50.]
 [  0.   0.   0.   0.]]
[453, 393, 523, 533]
===++++++++++======
59535.52238221133
logica
False False False False
523 533
matrizBinaria
[[0. 1. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]
matrizDeltaT
[[  0.  90.  20.  83.]
 [  0.   0. -10.  38.]
 [  0.   0.   0. -50.]
 [  0.   0.   0.   0.]]
[453, 393, 523, 533]
==xxxxxxxxxx=====
59535.52238221133
logica
True False False False
453 393
logica
False False False False
523 533
matrizBinaria
[[0. 1. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]
matrizDeltaT
[[  0. -90.  20.  83.]
 [  0.   0. -10.  38.]
 [  0.   0.   0. -50.]
 [  0.   0.   0.   0.]]
[453, 393, 523, 533]
==xxxxxxxxxx=====
59535.52238221133
(100, -100)
(100, -100)
(100, -100)
(100, -100)
(100, -100)
(100, -100)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)


In [0]:

#print(bounds)
print('temperatura atual=',tempAtual)
print('===========================')
print('===========================')
print('===========================')
fchoice=FUN
result = differential_evolution(fchoice, bounds, maxiter=ITE, popsize=NPAR)
print('temperatura atual=',tempAtual)
print(result.x, result.fun)
print(tempAtual)
GBEST=result.x
BEST=result.fun
matrizTrocaBest,matrizBinaria=MATBEST(GBEST,mCp,tempAtual)
matrizGrafico=MATGRAF(matrizTrocaBest,mCp,tempAtual)

print('Melhor solução para %d iterações com %d partículas \n',ITE,NPAR);print(GBEST)
print('Valor da função f(gbest)=%f \n',BEST)

print('Matriz de Trocas Térmicas \n')
print(matrizTrocaBest)
print('Matriz Binária \n')
print(matrizBinaria)


Streaming output truncated to the last 5000 lines.
logica
True False False False
453 393
matrizBinaria
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
matrizDeltaT
[[  0.         -83.70416662 -54.85681798  48.72039606]
 [  0.           0.          37.97524349  46.99741829]
 [  0.           0.           0.         -50.19134204]
 [  0.           0.           0.           0.        ]]
[453, 393, 523, 533]
logica
False False False False
453 533
matrizBinaria
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
matrizDeltaT
[[  0.          25.8462317  -79.84488751 -20.04318145]
 [  0.           0.          36.03633547   1.66012533]
 [  0.           0.           0.          -1.34103486]
 [  0.           0.           0.           0.        ]]
[453, 393, 523, 533]
matrizBinaria
[[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
matrizDeltaT
[[  0.         -55.05788236  30.27748754  92.84022157]
 [  0.           0.          26.83846904  11.01424936]
 [  0.        